In [6]:
from google.colab import drive
drive.mount('/content/mydrive')

In [82]:
%%capture
!pip install pytorch-lightning
!pip install timm
!pip install -U albumentations
!pip install -U opencv-python-headless==4.1.2.30

In [2]:
%%capture
!wget https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_GroundTruth.csv
!wget https://data.mendeley.com/public-files/datasets/zr7vgbcyr2/files/fa850265-57da-48f0-ba3e-998b3e44b1f6/file_downloaded -O pad.csv
!wget https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip

In [2]:
%%capture
!wget https://data.mendeley.com/public-files/datasets/zr7vgbcyr2/files/1cc2f71f-20a2-412d-b746-a9b9bc20c966/file_downloaded -O imgs_part_1.zip
!wget https://data.mendeley.com/public-files/datasets/zr7vgbcyr2/files/559a60ed-5504-475d-996c-6a8bc253b5e7/file_downloaded -O imgs_part_2.zip
!wget https://data.mendeley.com/public-files/datasets/zr7vgbcyr2/files/34dcdf8e-e5f1-4b35-aa0b-5135051ff852/file_downloaded -O imgs_part_3.zip


In [3]:
%%capture
!unzip /content/imgs_part_1.zip
!unzip /content/imgs_part_2.zip
!unzip /content/imgs_part_3.zip

In [4]:
!mkdir paddataset

In [5]:
!mv /content/imgs_part_1/*.png paddataset/
!mv /content/imgs_part_2/*.png paddataset/
!mv /content/imgs_part_3/*.png paddataset/

In [7]:
%%capture 
!unzip /content/ISIC_2019_Training_Input.zip

In [8]:
!rm /content/ISIC_2019_Training_Input.zip
!rm -rf /content/imgs_part_1
!rm -rf /content/imgs_part_2
!rm -rf /content/imgs_part_3
!rm /content/pad.zip

rm: cannot remove '/content/pad.zip': No such file or directory


In [9]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt

In [10]:
isic19=pd.read_csv('/content/ISIC_2019_Training_GroundTruth.csv')
isic19.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
isic19['label']=pd.Series(isic19.iloc[:,1::].idxmax(axis=1))

In [12]:
isic19.label=isic19.label.map({'NV':'NEV','AK':'ACK','BCC':'BCC','BKL':'BKL','SCC':'SCC','VASC':'VASC','DF':'DF','MEL':'MEL'})

In [13]:
isic19.drop(['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK'],axis=1,inplace=True)

In [14]:
isic19.head()

,image,label
0,ISIC_0000000,NEV
1,ISIC_0000001,NEV
2,ISIC_0000002,MEL
3,ISIC_0000003,NEV
4,ISIC_0000004,MEL


In [15]:
isic19.image='ISIC_2019_Training_Input/'+isic19.image+'.jpg'

In [16]:
pad=pd.read_csv('pad.csv')
pad=pad[['img_id','diagnostic']]
pad.columns=['image','label']
pad.image='paddataset/'+pad.image
pad.head()

,image,label
0,paddataset/PAT_1516_1765_530.png,NEV
1,paddataset/PAT_46_881_939.png,BCC
2,paddataset/PAT_1545_1867_547.png,ACK
3,paddataset/PAT_1989_4061_934.png,ACK
4,paddataset/PAT_684_1302_588.png,BCC


In [67]:
df=pd.concat([pad,isic19])
df.head()

,image,label
0,paddataset/PAT_1516_1765_530.png,NEV
1,paddataset/PAT_46_881_939.png,BCC
2,paddataset/PAT_1545_1867_547.png,ACK
3,paddataset/PAT_1989_4061_934.png,ACK
4,paddataset/PAT_684_1302_588.png,BCC


In [68]:
df=df[(df.label!='DF') & (df.label!='SEK') & (df.label!='VASC')].reset_index(drop=True)

In [69]:
df = df.drop(df[df['label'] =='NEV'].sample(frac=.5).index)


In [70]:
df.label.value_counts()

NEV    6559
MEL    4574
BCC    4168
BKL    2624
ACK    1597
SCC     820
Name: label, dtype: int64

In [71]:
df.label=df.label.map({v:k for k,v in enumerate(df.label.unique())})
                          

In [72]:
#import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pytorch_lightning import seed_everything, LightningModule, Trainer
from sklearn.utils import class_weight
import torch.nn as nn
import torch
from pytorch_lightning.callbacks import EarlyStopping,ModelCheckpoint,LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import torchvision
from sklearn.metrics import classification_report
from PIL import Image
from torch.utils.data import DataLoader, Dataset,random_split
import timm
import torchmetrics
import torchvision.models as models
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pytorch_lightning as pl
import sklearn
import cv2

In [73]:
print('torch version',torch.__version__)
print('pytorch lightnging version',pl.__version__)
print('sklearn version',sklearn.__version__)
print('torchvision version',torchvision.__version__)
print('albumentations version',A.__version__)
print('torchmetrics version',torchmetrics.__version__)
print('cv2 version',cv2.__version__)

torch version 1.10.0+cu111
pytorch lightnging version 1.5.10
sklearn version 1.0.2
torchvision version 0.11.1+cu111
albumentations version 1.1.0
torchmetrics version 0.7.2
cv2 version 4.1.2


In [74]:
img_size=224
from torchvision import transforms

aug=transforms.Compose([
            transforms.Resize(size=(224,224)),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ColorJitter(0.1,0.1,0,0),
            transforms.RandomRotation(180),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),
            ])



In [75]:
class DataReader(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __getitem__(self, index):
        x=self.dataset.image[index]#read image
        y=self.dataset.label[index] #read label
        image=Image.open(x).convert('RGB')
        if self.transform:#apply augmentations
            
            x=self.transform(image)
            
        return x, torch.tensor(y)
    
    def __len__(self):
        return len(self.dataset)

In [76]:
class OurModel(LightningModule):
    def __init__(self):
        super(OurModel,self).__init__()
        # model architecute
        '''refernce:https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/efficientnet.py'''
        self.model =  timm.create_model(model_name,pretrained=True)
        self.fc1=nn.Linear(1000,500)
        self.relu=nn.ReLU()
        self.fc2= nn.Linear(500,6)
 
        #parameters
        self.lr=1e-3
        self.batch_size=64
        self.numworker=2
        self.acc = torchmetrics.F1Score(num_classes=6,average='macro') #metric
        self.criterion=nn.CrossEntropyLoss() #loss function
        #list to store loss and accuracy
        self.trainacc,self.valacc=[],[]
        self.trainloss,self.valloss=[],[]
        #load data        
   
    def forward(self,x):
        x= self.model(x)
        x=self.fc1(x)
        x=self.relu(x)
        x=self.fc2(x)
        return x

    def mixup_data(self,x, y, alpha=1.0):
        '''
        Returns mixed inputs, pairs of targets, and lambda
        reference: mixup: Beyond Empirical Risk Minimization
        '''
        if alpha > 0:
            lam = np.random.beta(alpha, alpha)
        else:
            lam = 1

        batch_size = x.size()[0]
        index = torch.randperm(batch_size)
        mixed_x = lam * x + (1 - lam) * x[index, :]
        y_a, y_b = y, y[index]
        return mixed_x, y_a, y_b, lam


    def mixup_criterion(self, pred, y_a, y_b, lam):
        return lam * self.criterion(pred, y_a) + (1 - lam) * self.criterion(pred, y_b)
    
    
    def configure_optimizers(self):
        #optimizer and scheduler
        opt=torch.optim.AdamW(params=self.parameters(),lr=self.lr )
        scheduler=CosineAnnealingWarmRestarts(opt,T_0=10, T_mult=1, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt,'lr_scheduler':scheduler}

    def train_dataloader(self):#load train 
        return DataLoader(DataReader(train_set,aug), batch_size = self.batch_size, 
                          num_workers=self.numworker,
                          pin_memory=True,shuffle=True)

    def training_step(self,batch,batch_idx):
        image,label=batch
        mixed_x, y_a, y_b, lam=self.mixup_data(image,label)#apply mixup
        out = self(mixed_x)#pass images to model
        loss=self.mixup_criterion(out,y_a, y_b, lam) #calculate loss
        acc=self.acc(out,label)#calculate accuracy
        return {'loss':loss,'acc':acc}

    def training_epoch_end(self, outputs):
        #average loss and accuracy in all batches of train data
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.trainacc.append(acc)
        self.trainloss.append(loss)
        self.log('train_loss', loss)
        self.log('train_acc', acc)
        
    def val_dataloader(self):
        ds=DataLoader(DataReader(val_set,aug), batch_size = self.batch_size,
                      num_workers=self.numworker,pin_memory=True, shuffle=False)
        return ds

    def validation_step(self,batch,batch_idx):
        image,label=batch
        out=self(image)
        loss=self.criterion(out,label)
        acc=self.acc(out,label)
        return {'loss':loss,'acc':acc}

    def validation_epoch_end(self, outputs):
        loss=torch.stack([x["loss"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        acc=torch.stack([x["acc"] for x in outputs]).mean().detach().cpu().numpy().round(2)
        self.valacc.append(acc)
        self.valloss.append(loss)
        print('validation loss accuracy ',self.current_epoch,loss, acc)
        self.log('val_loss', loss)
        self.log('val_acc', acc)

In [77]:
from sklearn.model_selection import train_test_split
train_set,val_set=train_test_split(df,test_size=0.2)
train_set.reset_index(drop=True,inplace=True)
val_set.reset_index(drop=True,inplace=True)

In [83]:
model_name='mobilenetv3_rw'
model=OurModel()

In [84]:
lr_monitor = LearningRateMonitor(logging_interval='epoch')
checkpoint=ModelCheckpoint(dirpath='checkpoints',filename='file', monitor='val_acc', verbose=False, save_last=True, mode='max')
trainer = Trainer(max_epochs=30, auto_lr_find=False, auto_scale_batch_size=False,
                #deterministic=True,
                gpus=-1,precision=16,
                accumulate_grad_batches=1,
                stochastic_weight_avg=False,
                enable_progress_bar = False,
                num_sanity_val_steps=2,
                callbacks=[lr_monitor,checkpoint]
                 )

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | MobileNetV3      | 5.5 M 
1 | fc1       | Linear           | 500 K 
2 | relu      | ReLU             | 0     
3 | fc2       | Linear           | 3.0 K 
4 | acc       | F1Score          | 0     
5 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
6.0 M     Trainable params
0         Non-trainable params
6.0 M     Total params
11.967    Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


validation loss accuracy  0 1.88 0.04
validation loss accuracy  0 0.93 0.52
validation loss accuracy  1 0.87 0.54


In [ ]:
torch.save(model.state_dict(), '/content/mydrive/MyDrive/'+model_name+'_skin.pt')

filepath ='/content/mydrive/MyDrive/'+model_name+'_skin.onnx'
input_sample = torch.randn((1,3, 224,224))
model.to_onnx(filepath, input_sample,opset_version=11, export_params=True)

In [32]:
torch.save(model, '/content/mydrive/MyDrive/model_name+'_skin_model.pth')


Mounted at /content/mydrive
